In [2]:
#Setting up the code and installing required packages 
from util import *
from glob import glob
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import LineString, Polygon
pd.set_option("display.max_columns", None)

In [3]:
#Navigate to intersects shapefile and separates out the year, finds how many years from 1800 and how many years to 2100
gdf = gpd.read_file(f"Data/Merged Intersects_UniqueID/JackettIsland_Intersects.shp")
gdf["Date"] = pd.to_datetime(gdf.ShorelineI, dayfirst=True, format='mixed')
gdf["Year"] = gdf.Date.dt.year
gdf["YearsSinceBase"] = (gdf.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
gdf["YearsUntilFuture"] = (
    pd.Timestamp(2100, 1, 1) - gdf.Date
    ).dt.days / 365.25
gdf.Date = gdf.Date.astype(str)
gdf

,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,Unique_ID,Date,geometry,Year,YearsSinceBase,YearsUntilFuture
0,04/19/2017,1.0,-97.80,1.602805e+06,5.444480e+06,2.94,2.010001e+11,2017-04-19,POINT Z (1602805.403 5444479.642 0.000),2017,217.292266,82.702259
1,01/13/2000,1.0,-54.62,1.602842e+06,5.444502e+06,5.32,2.010001e+11,2000-01-13,POINT Z (1602842.047 5444502.478 0.000),2000,200.027379,99.967146
2,02/19/2010,1.0,-69.27,1.602830e+06,5.444495e+06,2.32,2.010001e+11,2010-02-19,POINT Z (1602829.612 5444494.729 0.000),2010,210.130048,89.864476
3,09/13/1985,1.0,-54.69,1.602842e+06,5.444502e+06,4.33,2.010001e+11,1985-09-13,POINT Z (1602841.988 5444502.442 0.000),1985,185.694730,114.299795
4,02/04/1967,1.0,-65.04,1.602833e+06,5.444497e+06,5.94,2.010001e+11,1967-04-02,POINT Z (1602833.209 5444496.970 0.000),1967,167.244353,132.750171
...,...,...,...,...,...,...,...,...,...,...,...,...
2337,02/22/1940,3.0,-29.51,1.603246e+06,5.444123e+06,3.77,2.010002e+11,1940-02-22,POINT Z (1603246.456 5444123.385 0.000),1940,140.136893,159.857632
2338,06/25/2013,3.0,-113.28,1.603204e+06,5.444051e+06,2.30,2.010002e+11,2013-06-25,POINT Z (1603203.881 5444051.243 0.000),2013,213.475702,86.518823
2339,08/28/2006,3.0,-109.38,1.603206e+06,5.444055e+06,2.32,2.010002e+11,2006-08-28,POINT Z (1603205.860 5444054.597 0.000),2006,206.650240,93.344285
2340,01/31/1980,3.0,-115.24,1.603203e+06,5.444050e+06,5.31,2.010002e+11,1980-01-31,POINT Z (1603202.885 5444049.556 0.000),1980,180.076660,119.917864


In [4]:
#Navigates to the rates shapefile
lines = gpd.read_file("Data/Merged rates_Transects/JackettIsland_rates_transects.shp")
lines

,ObjectID_1,BaselineID,TransOrder,TransEdit,TransectId,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,Azimuth,geometry
0,519.0,1.0,519.0,0.0,519.0,13.0,-0.16,0.07,0.32,16.77,-0.07,0.06,-5.38,44.09,-0.19,0.08,0.34,4.94,7410.848139,Tasman,238.069317,"LINESTRING Z (1603058.142 5444637.146 0.000, 1..."
1,520.0,1.0,520.0,0.0,520.0,13.0,-0.22,0.13,0.30,15.96,-0.12,0.06,-9.85,44.06,-0.25,0.15,0.32,4.70,7420.848139,Tasman,238.069318,"LINESTRING Z (1603052.853 5444645.633 0.000, 1..."
2,521.0,1.0,521.0,0.0,521.0,13.0,-0.26,0.19,0.28,15.15,-0.18,0.06,-14.58,44.19,-0.29,0.21,0.30,4.41,7430.848139,Tasman,238.069318,"LINESTRING Z (1603047.564 5444654.120 0.000, 1..."
3,522.0,1.0,522.0,0.0,522.0,13.0,-0.30,0.26,0.27,14.45,-0.24,0.06,-19.54,44.86,-0.34,0.29,0.28,4.13,7440.848139,Tasman,238.069318,"LINESTRING Z (1603042.275 5444662.606 0.000, 1..."
4,523.0,1.0,523.0,0.0,523.0,13.0,-0.34,0.33,0.26,14.09,-0.33,0.06,-26.59,45.63,-0.39,0.37,0.27,4.01,7450.848139,Tasman,238.069318,"LINESTRING Z (1603036.986 5444671.093 0.000, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,786.0,3.0,786.0,0.0,786.0,11.0,-0.99,0.82,0.28,13.37,-1.30,0.06,-100.43,100.43,-0.99,0.87,0.23,3.15,13980.696530,Tasman,222.273689,"LINESTRING Z (1603429.678 5444275.426 0.000, 1..."
179,787.0,3.0,787.0,0.0,787.0,11.0,-1.01,0.84,0.27,12.81,-1.33,0.06,-102.67,102.67,-1.01,0.88,0.22,3.06,13990.696530,Tasman,210.547570,"LINESTRING Z (1603388.941 5444305.792 0.000, 1..."
180,788.0,3.0,788.0,0.0,788.0,11.0,-0.99,0.83,0.28,13.26,-1.31,0.06,-101.16,101.16,-0.99,0.87,0.23,3.14,14000.696530,Tasman,210.547570,"LINESTRING Z (1603380.329 5444310.874 0.000, 1..."
181,789.0,3.0,789.0,0.0,789.0,11.0,-0.95,0.80,0.29,13.66,-1.30,0.06,-100.62,100.62,-0.95,0.85,0.24,3.27,14010.696530,Tasman,210.547570,"LINESTRING Z (1603371.717 5444315.957 0.000, 1..."


In [5]:
#Finding the azimuth of the transect lines in the rates shapefile and addding it as an attribute
def get_azimuth(line):
    p1, p2 = line.coords
    azimuth = math.degrees(math.atan2(p2[0]-p1[0], p2[1]-p1[1]))
    if azimuth < 0:
        azimuth += 360
    return azimuth
lines["Azimuth"] = lines.geometry.apply(get_azimuth)
lines.to_file("Data/Merged rates_Transects/JackettIsland_rates_transects.shp")
lines

,ObjectID_1,BaselineID,TransOrder,TransEdit,TransectId,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,Azimuth,geometry
0,519.0,1.0,519.0,0.0,519.0,13.0,-0.16,0.07,0.32,16.77,-0.07,0.06,-5.38,44.09,-0.19,0.08,0.34,4.94,7410.848139,Tasman,238.069317,"LINESTRING Z (1603058.142 5444637.146 0.000, 1..."
1,520.0,1.0,520.0,0.0,520.0,13.0,-0.22,0.13,0.30,15.96,-0.12,0.06,-9.85,44.06,-0.25,0.15,0.32,4.70,7420.848139,Tasman,238.069318,"LINESTRING Z (1603052.853 5444645.633 0.000, 1..."
2,521.0,1.0,521.0,0.0,521.0,13.0,-0.26,0.19,0.28,15.15,-0.18,0.06,-14.58,44.19,-0.29,0.21,0.30,4.41,7430.848139,Tasman,238.069318,"LINESTRING Z (1603047.564 5444654.120 0.000, 1..."
3,522.0,1.0,522.0,0.0,522.0,13.0,-0.30,0.26,0.27,14.45,-0.24,0.06,-19.54,44.86,-0.34,0.29,0.28,4.13,7440.848139,Tasman,238.069318,"LINESTRING Z (1603042.275 5444662.606 0.000, 1..."
4,523.0,1.0,523.0,0.0,523.0,13.0,-0.34,0.33,0.26,14.09,-0.33,0.06,-26.59,45.63,-0.39,0.37,0.27,4.01,7450.848139,Tasman,238.069318,"LINESTRING Z (1603036.986 5444671.093 0.000, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,786.0,3.0,786.0,0.0,786.0,11.0,-0.99,0.82,0.28,13.37,-1.30,0.06,-100.43,100.43,-0.99,0.87,0.23,3.15,13980.696530,Tasman,222.273689,"LINESTRING Z (1603429.678 5444275.426 0.000, 1..."
179,787.0,3.0,787.0,0.0,787.0,11.0,-1.01,0.84,0.27,12.81,-1.33,0.06,-102.67,102.67,-1.01,0.88,0.22,3.06,13990.696530,Tasman,210.547570,"LINESTRING Z (1603388.941 5444305.792 0.000, 1..."
180,788.0,3.0,788.0,0.0,788.0,11.0,-0.99,0.83,0.28,13.26,-1.31,0.06,-101.16,101.16,-0.99,0.87,0.23,3.14,14000.696530,Tasman,210.547570,"LINESTRING Z (1603380.329 5444310.874 0.000, 1..."
181,789.0,3.0,789.0,0.0,789.0,11.0,-0.95,0.80,0.29,13.66,-1.30,0.06,-100.62,100.62,-0.95,0.85,0.24,3.27,14010.696530,Tasman,210.547570,"LINESTRING Z (1603371.717 5444315.957 0.000, 1..."


In [6]:
#Function allows code to find the necessary information (Azimuth, how many intersect points along each transect) to run projection
def get_transect_metadata(transect_lines_shapefile: str):
    lines = gpd.read_file(transect_lines_shapefile)
    if "TransectID" in lines.columns:
        lines = lines.set_index("TransectID").sort_index()
    else:
        lines = lines.set_index("TransOrder").sort_index()
    lines["dist_to_neighbour"] = lines.distance(lines.shift(-1))
    breakpoints = lines.dist_to_neighbour[lines.dist_to_neighbour > 105]
    lines["group"] = pd.Series(range(len(breakpoints)), index=breakpoints.index)
    lines["group"] = lines.group.bfill().fillna(len(breakpoints)).astype(int)
    metadata = lines[["Azimuth", "group"]].to_dict(orient="index")
    return metadata

transect_metadata = get_transect_metadata(f"Data/Merged rates_Transects/JackettIsland_rates_transects.shp")
transect_metadata

{519.0: {'Azimuth': 238.06931706739624, 'group': 0},
 520.0: {'Azimuth': 238.06931847877917, 'group': 0},
 521.0: {'Azimuth': 238.0693184787536, 'group': 0},
 522.0: {'Azimuth': 238.06931847874912, 'group': 0},
 523.0: {'Azimuth': 238.06931847887205, 'group': 0},
 524.0: {'Azimuth': 238.06931847873466, 'group': 0},
 525.0: {'Azimuth': 238.06931847877354, 'group': 0},
 526.0: {'Azimuth': 238.06931847879298, 'group': 0},
 527.0: {'Azimuth': 238.06931847881177, 'group': 0},
 528.0: {'Azimuth': 238.0693184787565, 'group': 0},
 529.0: {'Azimuth': 238.06931847878468, 'group': 0},
 530.0: {'Azimuth': 238.0693184788987, 'group': 0},
 531.0: {'Azimuth': 238.06931847876416, 'group': 0},
 532.0: {'Azimuth': 238.0693184787853, 'group': 0},
 533.0: {'Azimuth': 238.06931847878081, 'group': 0},
 534.0: {'Azimuth': 237.70711291062696, 'group': 0},
 535.0: {'Azimuth': 237.707113473959, 'group': 0},
 536.0: {'Azimuth': 237.70711347436023, 'group': 0},
 537.0: {'Azimuth': 237.70711291025754, 'group': 0},

In [7]:
#Linear regression is run here. See util.py for the breakdown on linear_models
linear_models = fit(gdf, transect_metadata)
linear_models

KeyError: 'TransectID'

In [ ]:
#Only run if rolling average is needed. Otherwise SKIP THIS
#linear_models = fit(gdf, transect_metadata)
#rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
#linear_models.slope = rolled_slopes.slope
#linear_models.dropna(inplace=True)
#linear_models

In [ ]:
#Coordinates of the projected shoreline are plotted here

#Changed coordinate function by making old_x and old_y negative 
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

#Removed other model equations and changed Azimuth addtion from 180 to 360 deg
def predict(
    df: pd.DataFrame,
    linear_models: pd.DataFrame,
    transect_metadata: dict,
):
    """_summary_

    Args:
        df (pd.DataFrame): dataframe with columns: TransectID, Date, Distance, YearsSinceBase
        linear_models (pd.DataFrame): dataframe with columns: TransectID, slope, intercept
        transect_metadata (dict): dict lookup of TransectID to Azimuth & group
        
    Returns:
        pd.DataFrame: resulting prediction points for the year 2100
    """
    results = []
    for i, row in linear_models.iterrows():
        transect_ID = row.TransectID
        transect_df = df[df.TransectID == transect_ID]
        latest_row = transect_df[transect_df.Date == transect_df["Date"].max()].iloc[0]
        future_year = int(row.get("FUTURE_YEAR", FUTURE_YEAR))
        result = row.to_dict()
        result.update({
            "TransectID": transect_ID,
            "BaselineID": latest_row.BaselineID,
            "group": row.group,
            "Year": future_year,
            "ocean_point": calculate_new_coordinates(
                latest_row.geometry.x,
                latest_row.geometry.y,
                transect_metadata[transect_ID]["Azimuth"] + 180,
                500,
            ),
        })
        
        model = "linear"
        slope = row.slope
        intercept = row.intercept

        predicted_distance = slope * (future_year - 1800) + intercept
        distance_difference = latest_row.Distance - predicted_distance
        result[f"{model}_model_point"] = calculate_new_coordinates(
            latest_row.geometry.x,
            latest_row.geometry.y,
            transect_metadata[transect_ID]["Azimuth"],
            distance_difference,
        )
        result[f"{model}_model_predicted_distance"] = predicted_distance
        result[f"{model}_model_distance"] = distance_difference
        results.append(result)
    results = gpd.GeoDataFrame(results)
    return results

In [ ]:
#Projection file is created here with the stats and coordinate points in table format
results = predict(gdf, linear_models, transect_metadata)
results

In [ ]:
#Spatial reference added to the results
results.set_geometry("linear_model_point", inplace=True, crs=2193)
results

In [ ]:
#Line and polygon shapefiles are created here 
def prediction_results_to_line_polygon(results: gpd.GeoDataFrame):
    lines = []
    polygons = []
    for group_name, group_data in results.groupby(["BaselineID", "group"]):
        if len(group_data) > 1:
            # Convert the points to LineString
            line = LineString(list(group_data.geometry))
            lines.append(line)
            # Convert the points to a closed Polygon
            polygon = Polygon(list(group_data.geometry) + list(group_data.ocean_point)[::-1])
            polygons.append(polygon)
    lines = gpd.GeoSeries(lines, crs=2193)
    polygons = gpd.GeoSeries(polygons, crs=2193)
    return lines, polygons

In [ ]:
lines, poly = prediction_results_to_line_polygon(results)

In [ ]:
#Saving line and polygon projection file to Z drive. Change file location accordingly  
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Z:\DSAS\Region\Southland\BigBay\BigBay_projection_output_lines.shp")
poly.to_file("Z:\DSAS\Region\Southland\BigBay\BigBay_projection_output_polygon.shp")

In [ ]:
#Saving line and polygon projection file to folder in VS Code. Change file location accordingly
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("TEST\Southland_BigBay\BigBay_projection_output_lines.shp")
poly.to_file("TEST\Southland_BigBay\BigBay_projection_output_polygon.shp")

In [ ]:
#Quick visualisation of projected polygon and historic shorelines 
m = poly.explore(tiles="Esri.WorldImagery")
gpd.GeoDataFrame(results.drop(columns=["ocean_point", "linear_model_point"]), geometry=results.linear_model_point).explore(m=m)
gdf.explore("Year", legend=True, m=m)